In [19]:
from keras.applications import ResNet50
from keras.models import Sequential, Model
from keras.layers import Dense, Input

NUM_CLASSES = 6
input = Input(shape=(224, 224, 3))

# Still not talking about our train/test data or any pre-processing.
model = ResNet50(input_tensor=input, include_top = False, weights = 'imagenet', pooling='avg')

# Say not to train first layer (ResNet) model as it is already trained
model.trainable = False

In [20]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_8[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [21]:
x = model.output
x = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(model.input, x)

In [22]:
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_8[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [25]:
from keras import optimizers

sgd = optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(optimizer = sgd, loss = 'categorical_crossentropy', metrics = ['accuracy'])


In [27]:
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

image_size = 224

# preprocessing_function is applied on each image but only after re-sizing & augmentation (resize => augment => pre-process)
# Each of the keras.application.resnet* preprocess_input MOSTLY mean BATCH NORMALIZATION (applied on each batch) stabilize the inputs to nonlinear activation functions
# Batch Normalization helps in faster convergence
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   horizontal_flip=True,
                                   vertical_flip=True)

# flow_From_directory generates batches of augmented data (where augmentation can be color conversion, etc)
# Both train & valid folders must have NUM_CLASSES sub-folders
train_generator = data_generator.flow_from_directory(
        './data/Benchmarkimage_20200821/train',
        target_size=(image_size, image_size),
        batch_size=6,
        class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
        './data/Benchmarkimage_20200821/val/',
        target_size=(image_size, image_size),
        batch_size=6,
        class_mode='categorical') 


Found 103 images belonging to 6 classes.
Found 18 images belonging to 6 classes.


In [35]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [34]:
fit_history = model.fit_generator(
        train_generator,
        steps_per_epoch=5,
        epochs = 10,
        validation_data=validation_generator,
        validation_steps=5)

Epoch 1/10


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node functional_3/conv1_conv/Conv2D (defined at <ipython-input-31-4fea930115fe>:1) ]] [Op:__inference_train_function_51585]

Function call stack:
train_function
